In [6]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#Buscar simple Inputer


In [7]:
#----------------PREPROCESAMIENTO----------------
df_train_xgboost = df_train
df_test_xgboost = df_test

#df_train_xgboost = df_train_xgboost.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_train_xgboost = df_train_xgboost.fillna(value = {'ciudad': 0, 'provincia': 0, 'tipodepropiedad': 0})

df_train_xgboost = df_train_xgboost.drop(['ciudad','fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
df_test_xgboost = df_test_xgboost.drop(['ciudad','fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_train_xgboost = df_train_xgboost.fillna(value = {'provincia': 0, 'tipodepropiedad': 0})

print(df_train_xgboost.shape)
#XGBoost maneja los NaNs, ver despues si lo dejamos como esta

df_train_xgboost = pd.get_dummies(df_train_xgboost, drop_first=True)#, sparse = True)
df_test_xgboost = pd.get_dummies(df_test_xgboost, drop_first=True)#, sparse = True)

print(df_train_xgboost.shape)
precio = df_train_xgboost['precio']
df_train_xgboost.drop(labels=['precio'], axis=1,inplace = True)
df_train_xgboost.insert(len(df_train_xgboost.columns), 'Precio', precio)

X_train, y_train = df_train_xgboost.iloc[:,:-1],df_train_xgboost.iloc[:,-1]

df_test_xgboost['tipodepropiedad_Garage'] = 0
df_test_xgboost['tipodepropiedad_Hospedaje'] = 0
df_test_xgboost = df_test_xgboost[list(X_train.columns.values)]

#data_dmatrix = xgb.DMatrix(data=X,label=y)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1, learning_rate = 0.01,
#                max_depth = 3, gamma = 1, n_estimators = 1000)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.05,
#                max_depth = 7, min_child_weight = 4, n_estimators = 500, nthread = 4, subsample = 0.7, silent = 1)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.04,
                max_depth = 9, min_child_weight = 1, n_estimators = 700, nthread = 4, subsample = 0.7, silent = 1)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(df_test_xgboost)


res = pd.DataFrame(preds, index=df_test_xgboost.index, columns=['precio'])


(240000, 15)
(240000, 67)


In [8]:
res2 = res
res2['id'] = df_test_xgboost['id']
res = res.reset_index(drop=True)
res.head()
res.set_index('id', inplace = True)
#res.columns = ['id', 'target']
#res.set_index('id', inplace = True)
res.rename(columns={'precio':'target'}, inplace=True)
res.head()


res.to_csv("XGBoost_grupo3.csv", header=True)